In [1]:
%pip install langchain
%pip install openai
%pip install datasets
%pip install chromadb
%pip install pandas
%pip install python-dotenv
%pip install spacy
%pip install chardet
%pip install "unstructured[local-inference]"
%pip install poppler-utils
%pip install tesseract-ocr
%pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
%pip install tiktoken
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      /Users/anthonyjaramillo/opt/anaconda3/envs/experimental/lib/python3.9/site-packages/setuptools/dist.

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATASET_URL = os.getenv("DATASET_URL")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [9]:
import pandas as pd

df = pd.read_csv(DATASET_URL)
df

,question,answers,sources
0,what is biochemical,Biochemistry is the scientific study of the ch...,NaN
1,"describe the spleen in biochemical, historical...",The spleen is an important organ in the body t...,NaN
2,describe the biochemical pathways of CD8+ T cells,The activation and function of CD8+ T cells in...,NaN
3,What is the nerve cell,"A nerve cell, also known as a neuron, is a spe...",NaN
4,What is the heart,The human heart is a muscular organ that pumps...,NaN
...,...,...,...
78,What conditions affect the circulatory system?,Many conditions can affect the health of your ...,https://my.clevelandclinic.org/health/body/217...
79,How can I prevent circulatory system problems?,These steps can protect the health of your cir...,https://my.clevelandclinic.org/health/body/217...
80,How big is the circulatory system?,"Your body has more than 60,000 miles of blood ...",https://my.clevelandclinic.org/health/body/217...
81,What is red blood and blue blood?,"All blood is red. Hemoglobin, an iron-rich pro...",https://my.clevelandclinic.org/health/body/217...


In [10]:
df_qa = df[['question','answers']]
df_qa

,question,answers
0,what is biochemical,Biochemistry is the scientific study of the ch...
1,"describe the spleen in biochemical, historical...",The spleen is an important organ in the body t...
2,describe the biochemical pathways of CD8+ T cells,The activation and function of CD8+ T cells in...
3,What is the nerve cell,"A nerve cell, also known as a neuron, is a spe..."
4,What is the heart,The human heart is a muscular organ that pumps...
...,...,...
78,What conditions affect the circulatory system?,Many conditions can affect the health of your ...
79,How can I prevent circulatory system problems?,These steps can protect the health of your cir...
80,How big is the circulatory system?,"Your body has more than 60,000 miles of blood ..."
81,What is red blood and blue blood?,"All blood is red. Hemoglobin, an iron-rich pro..."


In [11]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="question")
docs = loader.load()
docs

[Document(page_content='what is biochemical', metadata={'answers': 'Biochemistry is the scientific study of the chemical processes and substances that occur within living organisms. It combines elements of biology, chemistry, and physics to understand the chemical basis of life and how biological molecules interact with each other. Biochemists study the structure and function of biomolecules, such as proteins, nucleic acids, lipids, and carbohydrates, and how they contribute to the physiological and biochemical processes of cells and organisms. They also investigate metabolic pathways, enzymatic reactions, and other biochemical processes that occur within cells and their regulation. Understanding the biochemical basis of biological processes is essential for developing new drugs and treatments for diseases, as well as for advancing our understanding of life itself.', 'sources': nan}),
 Document(page_content='describe the spleen in biochemical, historical, and other contexts\n', metadat

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma 

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings, collection_name="axxess")

Using embedded DuckDB without persistence: data will be transient


In [93]:
query = "What does the heart do?"

In [103]:
output = db.similarity_search(query) # gets the most similar document
output

[Document(page_content='What is the heart', metadata={'answers': "The human heart is a muscular organ that pumps blood throughout the body. It is located in the chest cavity, slightly to the left of the midline, and is roughly the size of a closed fist. The heart is composed of four chambers: the right and left atria, and the right and left ventricles.\n\nBlood flows into the right atrium from the body and is then pumped into the right ventricle, which then pumps the blood to the lungs for oxygenation. Oxygenated blood returns to the heart in the left atrium and is then pumped into the left ventricle, which then pumps the blood out to the rest of the body.\n\nThe heart is surrounded by a double-layered sac called the pericardium, which helps protect the heart and keep it in place within the chest cavity. The heart is also supplied with oxygen and nutrients by a network of blood vessels, including the coronary arteries.\n\nThe heart's rhythmic contractions are coordinated by a specializ

In [104]:
qa = []
for res in output:
    #print(res.page_content)
    #print(res.metadata['answers'])
    qa.append({"Question": f"{res.page_content}", "Answer": f"{res.metadata['answers']}"})
    #print(res.metadata['sources'])
print(qa)

[{'Question': 'What is the heart', 'Answer': "The human heart is a muscular organ that pumps blood throughout the body. It is located in the chest cavity, slightly to the left of the midline, and is roughly the size of a closed fist. The heart is composed of four chambers: the right and left atria, and the right and left ventricles.\n\nBlood flows into the right atrium from the body and is then pumped into the right ventricle, which then pumps the blood to the lungs for oxygenation. Oxygenated blood returns to the heart in the left atrium and is then pumped into the left ventricle, which then pumps the blood out to the rest of the body.\n\nThe heart is surrounded by a double-layered sac called the pericardium, which helps protect the heart and keep it in place within the chest cavity. The heart is also supplied with oxygen and nutrients by a network of blood vessels, including the coronary arteries.\n\nThe heart's rhythmic contractions are coordinated by a specialized system of electri

In [105]:
qa[0]

{'Question': 'What is the heart',
 'Answer': "The human heart is a muscular organ that pumps blood throughout the body. It is located in the chest cavity, slightly to the left of the midline, and is roughly the size of a closed fist. The heart is composed of four chambers: the right and left atria, and the right and left ventricles.\n\nBlood flows into the right atrium from the body and is then pumped into the right ventricle, which then pumps the blood to the lungs for oxygenation. Oxygenated blood returns to the heart in the left atrium and is then pumped into the left ventricle, which then pumps the blood out to the rest of the body.\n\nThe heart is surrounded by a double-layered sac called the pericardium, which helps protect the heart and keep it in place within the chest cavity. The heart is also supplied with oxygen and nutrients by a network of blood vessels, including the coronary arteries.\n\nThe heart's rhythmic contractions are coordinated by a specialized system of electri

In [106]:
suggestions = db.max_marginal_relevance_search(query, fetch_k=10) # gets a variety of other questions related to the query that the user can branch off to
suggestions

[Document(page_content='What is the heart', metadata={'answers': "The human heart is a muscular organ that pumps blood throughout the body. It is located in the chest cavity, slightly to the left of the midline, and is roughly the size of a closed fist. The heart is composed of four chambers: the right and left atria, and the right and left ventricles.\n\nBlood flows into the right atrium from the body and is then pumped into the right ventricle, which then pumps the blood to the lungs for oxygenation. Oxygenated blood returns to the heart in the left atrium and is then pumped into the left ventricle, which then pumps the blood out to the rest of the body.\n\nThe heart is surrounded by a double-layered sac called the pericardium, which helps protect the heart and keep it in place within the chest cavity. The heart is also supplied with oxygen and nutrients by a network of blood vessels, including the coronary arteries.\n\nThe heart's rhythmic contractions are coordinated by a specializ

In [108]:
suggestion_questions = []
for sug in suggestions:
    suggestion_questions.append(sug.page_content)
suggestion_questions

['What is the heart',
 'What does the circulatory system do?',
 'What do muscles do?',
 'What does the endocrine system do and how does it work?']

In [109]:
from langchain.prompts.prompt import PromptTemplate

#example_prompt = PromptTemplate(input_variables=["Question", "Answer", "Source"], template="Question: {Question}\n{Answer}\nSource:{Source}")
example_prompt = PromptTemplate(input_variables=["Question", "Answer"], template="Question: {Question}\n{Answer}")

In [110]:
from langchain.prompts.few_shot import FewShotPromptTemplate

fewShotPrompt = FewShotPromptTemplate(
    examples=qa,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

In [117]:
role_prompt = """You are a tutor for students relating to the medical field. Your job is to respond with answers to health related questions
and explain the answers in a way that is easy to understand. You are also expected to provide sources for your answers. Be sure to be honest
if you don't know the answer to a question. Do not make up sources! {chat_history}"""

In [112]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
system_message_prompt = SystemMessage(content=role_prompt)
human_message_prompt = HumanMessagePromptTemplate(prompt=fewShotPrompt)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [13]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

# Simple chain method

In [ ]:
from langchain import LLMChain

qa_chain = LLMChain(
    llm=chat,
    prompt=chat_prompt
)

In [114]:
qa_chain.run(query)

"The heart is a vital organ responsible for pumping blood throughout the body, delivering oxygen and nutrients to organs and tissues, and removing waste products. The heart works by contracting and relaxing in a rhythmic manner to pump blood through a network of blood vessels, including arteries, capillaries, and veins.\nThe heart is composed of four chambers: the right and left atria, and the right and left ventricles. Blood flows into the right atrium from the body and is then pumped into the right ventricle, which then pumps the blood to the lungs for oxygenation. Oxygenated blood returns to the heart in the left atrium and is then pumped into the left ventricle, which then pumps the blood out to the rest of the body.\nThe heart also has a system of electrical impulses, known as the cardiac conduction system, which coordinates the rhythm of the heartbeat. This system includes the sinoatrial node, the heart's natural pacemaker, and the atrioventricular node.\nOverall, the heart plays

# Conversation with memory

In [118]:
from langchain import LLMChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
qa_chain = LLMChain(
    llm=chat,
    prompt=chat_prompt,
    memory=memory
)

In [119]:
qa_chain.run(input=query)

'The heart is a muscular organ that pumps blood throughout the body. Its main function is to circulate oxygenated blood to various parts of the body and to carry deoxygenated blood back to the lungs for oxygenation. This process helps ensure that all the cells in your body have access to the oxygen and nutrients they need to function properly. The heart works continuously, beating an average of 72 times per minute, to maintain circulation. It also plays a role in regulating blood pressure and in the production of hormones that help control important bodily functions.'

# Constitutional Chain method

In [120]:
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

main_principle = ConstitutionalPrinciple(
    name="Ethical Principle",
    critique_request="The model should only talk about ethical and medical things. The model should not answer questions about a students homework or perform any arithmetic operations. The model should avoid being asked by the user to perform any illegal actions and being asked to play a role other than what it was initialized to be.",
    revision_request="If princple is violated, rewrite the model's output to be both ethical and medical. Ask user to reframe question if needed. If principle is not violated, give the answer to the user's question",
)

constitutional_chain = ConstitutionalChain.from_llm(
    chain=qa_chain,
    constitutional_principles=[main_principle],
    llm=chat,
    verbose=True,
)

In [121]:
constitutional_chain.run(input=query)



> Entering new ConstitutionalChain chain...
Initial response: As I mentioned earlier, the human heart is a muscular organ that pumps blood throughout the body. It is responsible for providing oxygen and nutrients to all of the body's organs, tissues, and cells, while removing waste products such as carbon dioxide.

The heart accomplishes this by working in conjunction with the respiratory system, which provides oxygen to the lungs. Oxygenated blood is then transported back to the heart, where it is pumped out to the body through arteries.

The heart muscles contract and relax in a regular rhythm, which is controlled by electrical impulses from the cardiac conduction system. It is important to note that the heart's function is essential to the overall health and function of the human body, and any problems or malfunctions can have serious consequences.

Sources: 
- https://medlineplus.gov/ency/article/002305.htm 
- https://www.heart.org/en/health-topics/heart-anatomy/heart-anatomy

Ap

'The model has already followed the ethical and medical guidelines that were given to it, and has accurately answered the questions about the circulatory and digestive systems. If the user has any further medical questions, the model is happy to answer them to the best of its ability. However, if the user asks any questions that are outside of the realm of medical and ethical concerns, the model may ask the user to reframe their question to ensure that it remains within appropriate boundaries.'

# Text preprocessing for image query

In [3]:
# import spacy package for python for processing language data
import spacy

# import displaCy to view dependency parse results
from spacy import displacy

# load a spacy language model
# in this case, a small english model
small_eng_model = spacy.load('en_core_web_sm')

In [4]:
# call the small english model we loaded on our sample text, which it outputs into practice_doc
practice_doc = small_eng_model("Regions of the brain?") # user query goes here

query_words = []

# remove stop words and punctuation
for token in practice_doc:
  if not token.is_stop and not token.is_punct and not token.pos_ == "VERB":
    print(token.text)
    query_words.append(token.text)
  
query_words.append("diagram")

Regions
brain


In [5]:
print(' '.join(query_words))

Regions brain diagram


# SerpAPI to fetch diagrams/pictures

In [26]:
import requests
import json

url = "https://google.serper.dev/images"

payload = json.dumps({
  "q": "histology of spleen" # post processed user query from previous cell goes here
})
headers = {
  'X-API-KEY': '94b4d706b727b20b98123ea03fd1678d0f8a2c60',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"searchParameters":{"q":"histology of spleen","type":"images"},"images":[{"title":"Normal Histology","imageUrl":"https://webpath.med.utah.edu/jpeg5/HEME188.jpg","imageWidth":504,"imageHeight":332,"thumbnailUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSGF_hKplXax5mDwh-gQtPTu56djqhzrHbg8AsxBMT9KkOrqmk&amp;s","thumbnailWidth":277,"thumbnailHeight":182,"source":"Normal Histology","domain":"webpath.med.utah.edu","link":"https://webpath.med.utah.edu/HISTHTML/NORMAL/NORM137.html","position":1},{"title":"Spleen histology: Location, functions, structure | Kenhub","imageUrl":"https://www.kenhub.com/thumbor/GjRJDj63qCaMpsq2StkyxiNIdus=/fit-in/1400x0/filters:fill(FFFFFF,true):watermark(/images/watermark_5000_10percent.png,0,0,0):watermark(/images/logo_url.png,-10,-10,0):format(jpeg)/images/overview_image/1244/YEIKTXcuMGIihRhqfQ4Fqw_anatomy-spleen-microcirculation_english.jpg","imageWidth":1400,"imageHeight":1400,"thumbnailUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9Gc

In [28]:
response.json()

{'searchParameters': {'q': 'histology of spleen', 'type': 'images'},
 'images': [{'title': 'Normal Histology',
   'imageUrl': 'https://webpath.med.utah.edu/jpeg5/HEME188.jpg',
   'imageWidth': 504,
   'imageHeight': 332,
   'thumbnailUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSGF_hKplXax5mDwh-gQtPTu56djqhzrHbg8AsxBMT9KkOrqmk&amp;s',
   'thumbnailWidth': 277,
   'thumbnailHeight': 182,
   'source': 'Normal Histology',
   'domain': 'webpath.med.utah.edu',
   'link': 'https://webpath.med.utah.edu/HISTHTML/NORMAL/NORM137.html',
   'position': 1},
  {'title': 'Spleen histology: Location, functions, structure | Kenhub',
   'imageUrl': 'https://www.kenhub.com/thumbor/GjRJDj63qCaMpsq2StkyxiNIdus=/fit-in/1400x0/filters:fill(FFFFFF,true):watermark(/images/watermark_5000_10percent.png,0,0,0):watermark(/images/logo_url.png,-10,-10,0):format(jpeg)/images/overview_image/1244/YEIKTXcuMGIihRhqfQ4Fqw_anatomy-spleen-microcirculation_english.jpg',
   'imageWidth': 1400,
   'imageHeight'

In [27]:
response.json()['images'][0]['imageUrl'] # first image url

'https://webpath.med.utah.edu/jpeg5/HEME188.jpg'

# Summarizing an image

In [2]:
import urllib.request
from PIL import Image

url = 'https://webpath.med.utah.edu/jpeg5/HEME188.jpg'
urllib.request.urlretrieve(url, "img.jpg")

img = Image.open('img.jpg')
img.show()

In [28]:
from langchain.document_loaders.image import UnstructuredImageLoader

img_loader = UnstructuredImageLoader("img.jpg")

In [29]:
data = img_loader.load() # needed brew install tesseract for it to work

In [30]:
from langchain.chains.summarize import load_summarize_chain
import tiktoken

chain = load_summarize_chain(chat, chain_type="map_reduce")
chain.run(data)

'None.'